In [ ]:
#@title Setup + Download Model
# Install and load dependencies
!pip install -q aitextgen
!pip install pytorch-lightning==1.7.0

import sys
from jinja2 import Template
from aitextgen import aitextgen

try:
    from google.colab import files
except ImportError:
    pass

# Download and load the model. Set to_gpu=False if running on a CPU.
ai = aitextgen(model="minimaxir/magic-the-gathering", to_gpu=False)

# This template is similar to Scryfall card formatting
TEMPLATE = Template(
    """{{ c.name }}{% if c.manaCost %} {{ c.manaCost }}{% endif %}
{{ c.type }}
{{ c.text }}{% if c.power %}
{{ c.power }}/{{ c.toughness }}{% endif %}{% if c.loyalty %}
Loyalty: {{ c.loyalty }}{% endif %}"""
)

def render_card(card_dict):
    card = TEMPLATE.render(c=card_dict)
    if card_dict["name"]:
        card = card.replace("~", card_dict["name"])
    return card

## Generating Cards

The following cell generates 8 unique cards each time it is run. But you can customize it to generate what you want!

The `prompt` is what guides the generation of the card. The model is trained on lightly-encoded variants of cards, with the following sections/keys: `<|name|>`, `<|manaCost|>`, `<|type|>`, `<|text|>`, `<|power|>`, `<|toughness|>`, `<|loyalty|>`. These keys are randomized, allowing the cards to be generated in any order. Therefore, you can prompt specific aspects of each card and the model will attempt to generate the rest.

Examples:

- A creature with a specific card name: `<|name|>Chaos Angel<|type|>Creature`
- A Jace Planeswalker: `<|type|>Legendary Planeswalker — Jace<|name|>Jace`
- A 5-Color Enchantment: `<|manaCost|>{W}{U}{B}{R}{G}<|type|>Enchantment`
- A win condition: `<|text|>You win the game if`
- A hybrid card: `<|manaCost|>{R/W}{R/W}{R/W}`
- A Creature with varying P/T: `<|power|>*<|toughness|>*<|type|>Creature`
- A coin flip card: `<|type|>Instant<|text|>Flip a coin. If`
- A Sorcery with Kicker: `<|type|>Sorcery<|text|>Kicker`
- A Planeswalker with low loyalty: `<|loyalty|>1<|text|>+1: Draw three cards`

The possibilities are endless!

The `temperature` controls the craziness of the generation. `0.1` will result in mostly realistic generation, `0.7` (default) is a happy medium between accuracy and creativity, `1.2` results in anarchy.

Checking the `to_file` checkbox will instead save 100 generated cards to `cards.txt` and download it, so you can filter through them offline.

Have fun!

**Important Note**: Not all generated cards will be legal or coherent, especially with narrow/unrealistic prompts and/or high temperatures. This is expected, and some level of curation of good card output is required.

In [ ]:
prompt="\u003C|type|>Creature - Human \u003C|name|>Rezo the Destroyer of Politics \u003C|manaCost|> 2 {B/G}" #@param {type:"string"}
temperature = 0.7 #@param {type:"slider", min:0.1, max:1.2, step:0.1}
to_file = False #@param {type:"boolean"}

n = 100 if to_file else 8

cards = ai.generate(n=n,
                    schema=True,
                    prompt=prompt,
                    temperature=temperature,
                    return_as_list=True)

cards = list(map(render_card, cards))

if to_file:
  file_path = "cards.txt"
  with open(file_path, "w", encoding="utf-8") as f:
    for card in cards:
      f.write("{}\n{}".format(card, "=" * 20 + "\n"))
  if "google.colab" in sys.modules:
    files.download(file_path)
else:
  print(("\n" + "=" * 20 + "\n").join(cards))

In [1]:
def generate_cards(
    n_cards: int = 8,
    temperature: float = 0.75,
    name: str = "",
    manaCost: str = "",
    type: str = "",
    text: str = "",
    power: str = "",
    toughness: str = "",
    loyalty: str = ""
):
    #ensure n_cards is never 0 or negative
    n_cards = int(n_cards)
    if n_cards < 1:
        n_cards = 1
    

    #change manaCost from Format:
    # 2UG
    #to:
    #{2}{U}{G}
    
    manaCost_str = ""

    for char in manaCost:
        manaCost_str += "{"
        manaCost_str += char
        manaCost_str += "}"
    
    


    prompt_str = ""

    token_dict = {
        "<|name|>": name,
        "<|manaCost|>": manaCost_str,
        "<|type|>": type,
        "<|text|>": text,
        "<|power|>": power,
        "<|toughness|>": toughness,
        "<|loyalty|>": loyalty
    }

    # Convert the token_dict into a formatted prompt string
    for token, value in token_dict.items():
        if value:
            prompt_str += f"{token}{value}"
    
    # Generate the cards using the prompt string and other parameters
    cards = ai.generate(
        n=n_cards,
        schema=True,
        prompt=prompt_str,
        temperature=temperature,
        return_as_list=True
    )

    cards = list(map(render_card, cards))

    return cards

In [ ]:
%pip install gradio
import gradio as gr

iface = gr.Interface(
    fn = generate_cards,
    inputs=[
        "Number",
        gr.Slider(minimum = 0.1, maximum=2.9, step=0.01, value=0.75),
        "Text",
        "Text",
        "Text",
        "Text",
        "Text",
        "Text",
        "Text"
    ],
    outputs="Text"
)
iface.launch()